Download this notebook in .ipynb format <a href='download_data_via_api.ipynb'>here</a>.

In [1]:
output_dir = '.'

In [3]:
import json
from pathlib import Path

from allensdk.api.api import Api
from allensdk.api.queries.rma_api import RmaApi
from allensdk.api.queries.mouse_connectivity_api import MouseConnectivityApi

rma = RmaApi()

def read_data(parsed_json):
    return parsed_json['msg']

def pretty(result):
    print(json.dumps(result, indent=2))

Download lists of experiments by injection hemisphere, mouse cre line, and injection structure

    Download Pvalb experiments injected in the left hemisphere VISp

In [4]:
# Get the atlas id
def query_atlases(search_pattern):
    return rma.build_query_url(rma.model_stage('Atlas',
                                               criteria="[name$il'%s']" % (search_pattern),
                                               only=['id', 'name']))

atlases = rma.do_query(query_atlases, read_data, 'mouse*')
pretty(atlases)


[
  {
    "id": 5,
    "name": "Mouse Sagittal, v1 (2008)"
  },
  {
    "id": 602630314,
    "name": "Mouse, Adult, 3D Coronal"
  },
  {
    "id": 2,
    "name": "Mouse, P56, Sagittal"
  },
  {
    "id": 4,
    "name": "Mouse Coronal, v1 (2008)"
  },
  {
    "id": 1,
    "name": "Mouse, P56, Coronal"
  }
]


In [5]:
# get the structure
def query_structure(acronym, ontology_id):
    return rma.build_query_url(rma.model_stage('Structure',
                                               criteria="[acronym$eq'%s'][ontology_id$eq%d]" % (acronym, ontology_id),
                                               only=['id','name']))

structure = rma.do_query(query_structure, read_data, 'VISp', 1)[0]
pretty(structure)


{
  "id": 385,
  "name": "Primary visual area"
}


In [6]:
def query_hemisphere(name):
    return rma.build_query_url(rma.model_stage('Hemisphere', criteria="[name$il'%s']" % (name)))
                               
left_hemisphere_id = rma.do_query(query_hemisphere, read_data, 'left')[0]['id']

left_hemisphere_id

1

In [7]:
mca = MouseConnectivityApi('http://api.brain-map.org')
experiments = mca.get_experiments(structure['id'])

# TODO: figure out why this didn't work w/ left hemisphere
#other_hemisphere_id = 3

# get experiments doesn't take hemisphere into account, so filter the results with a list comprehension
#left_hemisphere_experiments = [e for e in experiments
#                               if any([(injection['primary_injection_structure']['hemisphere_id'] == other_hemisphere_id)
#                               for injection in e['specimen']['stereotaxic_injections']])]
                                                          
#pretty(left_hemisphere_experiments)
len(experiments)

0

Download grid data for these experiments at a particular resolution

In [8]:
from allensdk.api.queries.grid_data_api import GridDataApi

10, 25, 50, or 100 micron volumes for:

    * Projection density

    * projection energy

    * injection fraction

    * Injection density

    * Injection energy

    * Data mask (mask of valid voxels per experiment)


In [9]:
gda = GridDataApi()

# TODO: show search to get this
section_data_set_id = 183282970

image_list = ['projection_density',
              'projection_energy',
              'injection_fraction',
              'injection_density',
              'injection_energy']

resolution = 100 # or 10, 25, 50

# Hmm, this didn't work for an image list of length > 1
for image in image_list:
    nrrd_file = str(Path(output_dir) / f'grid_{section_data_set_id}_{image}.nrrd')
    gda.download_projection_grid_data(section_data_set_id,
                                      image=[image],
                                      resolution=resolution,
                                      save_file_path=nrrd_file)


# TODO: data mask

2022-12-09 16:44:56,577 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/183282970?image=projection_density&resolution=100
2022-12-09 16:44:57,742 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/183282970?image=projection_energy&resolution=100
2022-12-09 16:44:58,300 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/183282970?image=injection_fraction&resolution=100
2022-12-09 16:44:58,609 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/183282970?image=injection_density&resolution=100
2022-12-09 16:44:58,744 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download_file/183282970?image=injection_energy&resolution=100


    http://help.brain-map.org//display/mouseconnectivity/API#API-Expression3DGrids

In [10]:
gda.download_expression_grid_data(
    section_data_set_id, 
    include=image_list,
    path=str(Path(output_dir) / f'{section_data_set_id}.zip')
)

/tmp/ipykernel_1995/1972217619.py:1: VisibleDeprecationWarning: Function download_expression_grid_data is deprecated. Use download_gene_expression_grid_data instead
  gda.download_expression_grid_data(
2022-12-09 16:44:58,954 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/grid_data/download/183282970?include=projection_density,projection_energy,injection_fraction,injection_density,injection_energy


Read data (File mode):

    Read experiment metadata by injection hemisphere, mouse cre line, and injection structure
        “I download a list of all experiments.  Which were injected in VISp?”

In [11]:
#http://api.brain-map.org/api/v2/data/query.csv?criteria=
#model::ProjectionStructureUnionize,
#rma::criteria,[is_injection$eq'f'],hemisphere,structure,section_data_set[id$eq183282970](specimen(stereotaxic_injections(primary_injection_structure,stereotaxic_injection_coordinates))),rma::include,section_data_set(specimen(stereotaxic_injections(primary_injection_structure))),
#rma::options[tabular$eq'distinct+specimens.name+as+specimen_name,stereotaxic_injection_coordinates.coordinates_ap,stereotaxic_injection_coordinates.coordinates_dv,stereotaxic_injection_coordinates.coordinates_ml,data_sets.id+as+data_set_id,stereotaxic_injections.primary_injection_structure_id,structures.acronym+as+target_structure,hemispheres.symbol+as+hemisphere,projection_structure_unionizes.is_injection,projection_structure_unionizes.sum_pixels,projection_structure_unionizes.sum_projection_pixels,projection_structure_unionizes.sum_pixel_intensity,projection_structure_unionizes.sum_projection_pixel_intensity,projection_structure_unionizes.projection_density,projection_structure_unionizes.projection_intensity,projection_structure_unionizes.projection_energy,projection_structure_unionizes.volume,projection_structure_unionizes.projection_volume,projection_structure_unionizes.normalized_projection_volume,projection_structure_unionizes.max_voxel_density,projection_structure_unionizes.max_voxel_x,projection_structure_unionizes.max_voxel_y,projection_structure_unionizes.max_voxel_z'][start_row$eq0][num_rows$eq3000]

def build_query(section_data_set_id):
    criteria_string = ''.join(["[is_injection$eq'f'],",
                               "hemisphere,",
                               "structure,",
                               "section_data_set[id$eq%d]" % (section_data_set_id),
                               "(specimen",
                               "(stereotaxic_injections",
                               "(primary_injection_structure,stereotaxic_injection_coordinates)",
                               "))"])
    include_string = ''.join(["section_data_set",
                              "(specimen",
                              "(stereotaxic_injections",
                              "(primary_injection_structure)",
                              "))"])
    tabular_list = ['distinct+specimens.name+as+specimen_name',
                    'stereotaxic_injection_coordinates.coordinates_ap',
                    'stereotaxic_injection_coordinates.coordinates_dv',
                    'stereotaxic_injection_coordinates.coordinates_ml',
                    'data_sets.id+as+data_set_id',
                    'stereotaxic_injections.primary_injection_structure_id',
                    'structures.acronym+as+target_structure',
                    'hemispheres.symbol+as+hemisphere',
                    'projection_structure_unionizes.is_injection',
                    'projection_structure_unionizes.sum_pixels',
                    'projection_structure_unionizes.sum_projection_pixels',
                    'projection_structure_unionizes.sum_pixel_intensity',
                    'projection_structure_unionizes.sum_projection_pixel_intensity',
                    'projection_structure_unionizes.projection_density',
                    'projection_structure_unionizes.projection_intensity',
                    'projection_structure_unionizes.projection_energy',
                    'projection_structure_unionizes.volume',
                    'projection_structure_unionizes.projection_volume',
                    'projection_structure_unionizes.normalized_projection_volume',
                    'projection_structure_unionizes.max_voxel_density',
                    'projection_structure_unionizes.max_voxel_x',
                    'projection_structure_unionizes.max_voxel_y',
                    'projection_structure_unionizes.max_voxel_z']
    model_stage = rma.model_stage('ProjectionStructureUnionize',
                                  criteria=criteria_string,
                                  include=include_string,
                                  tabular=["'%s'" % ','.join(tabular_list)],  # TODO: better handling of tabular quotes
                                  num_rows='all')
    url = rma.build_query_url(model_stage, fmt='csv')
    
    return url

print(build_query(183282970))
    


http://api.brain-map.org/api/v2/data/query.csv?q=model::ProjectionStructureUnionize,rma::criteria,[is_injection$eq'f'],hemisphere,structure,section_data_set[id$eq183282970](specimen(stereotaxic_injections(primary_injection_structure,stereotaxic_injection_coordinates))),rma::include,section_data_set(specimen(stereotaxic_injections(primary_injection_structure))),rma::options[tabular$eq'distinct+specimens.name+as+specimen_name,stereotaxic_injection_coordinates.coordinates_ap,stereotaxic_injection_coordinates.coordinates_dv,stereotaxic_injection_coordinates.coordinates_ml,data_sets.id+as+data_set_id,stereotaxic_injections.primary_injection_structure_id,structures.acronym+as+target_structure,hemispheres.symbol+as+hemisphere,projection_structure_unionizes.is_injection,projection_structure_unionizes.sum_pixels,projection_structure_unionizes.sum_projection_pixels,projection_structure_unionizes.sum_pixel_intensity,projection_structure_unionizes.sum_projection_pixel_intensity,projection_structur

    Read experiment grid data for these experiments by type
        3D numpy arrays worked pretty well for Friday harbor last year

Process data:

    Mask grid data using the data mask or injection fraction volumes

        “Give me the projection energy voxels from this experiment that are valid for analysis”

        “Give me the injection density voxels inside the injection site”

    Mask grid data arrays by structure

        “Give me the projection density voxel data for voxels belonging to VISp”

        This will involve the annotation volume at the correct resolution
            http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2015/


In [12]:
resolution=25
mca.download_volumetric_data(
    'annotation/ccf_2015', 
    'annotation_%d.nrrd' % (resolution),
    save_file_path=str(Path(output_dir) / 'volumetric_data.nrrd')
)


2022-12-09 16:44:59,985 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2015/annotation_25.nrrd
